In [3]:
pip install -U fiftyone

In [10]:
import fiftyone.utils.huggingface as fouh

dataset = fouh.load_from_hub("Voxel51/VisDrone2019-DET")

INFO:fiftyone.utils.huggingface:Downloading config file fiftyone.yml from Voxel51/VisDrone2019-DET


Loading dataset


INFO:fiftyone.utils.huggingface:Loading dataset


ValueError: Dataset name 'Voxel51/VisDrone2019-DET' is not available

In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

In [ ]:
import fiftyone as fo

datasets = fo.list_datasets()
print("Existing Datasets:", datasets)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import fiftyone as fo
dataset = fo.load_dataset("Voxel51/VisDrone2019-DET")  #"Voxel51/VisDrone2019-DET"

In [ ]:
print(f"Dataset name: {dataset.name}")
print(f"Number of samples: {len(dataset)}")

for sample in dataset.take(5):
    print(sample.id)


In [ ]:
first_sample = dataset.first()
field_names = first_sample.to_dict().keys()

print("Field names in the dataset:", list(field_names))


In [ ]:
for sample in dataset:
    image_path = sample.filepath
    print(f"Image Path: {image_path}")

    if sample.ground_truth is not None:
        detections = sample.ground_truth.detections
        for detection in detections:
            print(f"Label: {detection.label}, Bounding Box: {detection.bounding_box}")
    else:
        print("No ground truth annotations available.")


In [ ]:
import cv2

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image at {image_path}. Skipping...")
        return None

    img = cv2.resize(img, (300, 300))
    img = img / 255.0
    return img

images = []
labels = []

for sample in dataset:
    image_path = sample.filepath
    img = preprocess_image(image_path)

    if img is not None:
        images.append(img)

        if sample.ground_truth is not None:
            detections = sample.ground_truth.detections
            label = [detection.label for detection in detections]
            labels.append(label)
        else:
            labels.append([])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(labels)

print(f"Encoded labels shape: {y_encoded.shape}")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}")


In [ ]:
model = models.Sequential([
    layers.Input(shape=(300, 300, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(y_encoded.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

In [ ]:
predictions = model.predict(X_test)

for i in range(5):
    print(f"Predicted label: {predictions[i]}")
